<a href="https://colab.research.google.com/github/Strikerzee/CS308-Sept-2020-Git-Main-Project/blob/master/LAPSlotassignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod
import pandas as pd
import math

In [ ]:
V = TypeVar('V') # variable type
D = TypeVar('D') # domain type

In [ ]:
# Base class for all constraints
class Constraint(Generic[V, D], ABC):
    # The variables that the constraint is between
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    # Must be overridden by subclasses
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        ...

In [ ]:
# A constraint satisfaction problem consists of variables of type V
# that have ranges of values known as domains of type D and constraints
# that determine whether a particular variable's domain selection is valid
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables # variables to be constrained
        self.domains: Dict[V, List[D]] = domains # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it.")
        self.slots: Dict[D, int] = {}
        self.slots["A"] = 0
        self.slots["B"] = 0
        self.slots["C"] = 0
        self.slots["D"] = 0
        self.slots["E"] = 0
        self.slots["F"] = 0

    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True

    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        # assignment is complete if every variable is assigned (our base case)
        if len(assignment) == len(self.variables):
            return assignment

        # get all variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        # get the every possible domain value of the first unassigned variable
        first: V = unassigned[0]
        for value in self.domains[first]:
            if self.slots[value] < 9:
                self.slots[value] = self.slots[value] + 1
                local_assignment = assignment.copy()
                local_assignment[first] = value
                # if we're still consistent, we recurse (continue)
                if self.consistent(first, local_assignment):
                    result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                    # if we didn't find the result, we will end up backtracking
                    if result is not None:
                        return result
                self.slots[value] = self.slots[value] - 1
        return None

In [ ]:
class CourseSchedulingConstraint(Constraint[str, str]):
    def __init__(self, course1: str, course2: str) -> None:
        super().__init__([course1, course2])
        self.course1: str = course1
        self.course2: str = course2

    def satisfied(self, assignment: Dict[str, str]) -> bool:
        # If either place is not in the assignment then it is not
        # yet possible for their colors to be conflicting
        if self.course1 not in assignment or self.course2 not in assignment:
            return True
        # check the color assigned to course1 is not the same as the
        # color assigned to course2
        return assignment[self.course1] != assignment[self.course2]

In [ ]:
if __name__ == "__main__":
    df= pd.read_csv('/content/Course List - Sheet1.csv', usecols= ['Course'], keep_default_na=False)
    df_course= pd.read_csv('/content/Course List - Sheet1.csv', keep_default_na=False)

    course_list = df.values.tolist()
    course_list = [item[0] for item in course_list]
    course_instructor_list = df_course.values.tolist()

    df_clash= pd.read_csv('/content/Course List - Sheet2.csv', keep_default_na=False)
    clash_list = df_clash.values.tolist()

    variables: List[str] = course_list

    domains: Dict[str, List[str]] = {}
    for variable in variables:
        domains[variable] = ["A", "B", "C", "D", "E", "F"]

    csp: CSP[str, str] = CSP(variables, domains)
    csp.add_constraint(CourseSchedulingConstraint("HS102", "HS103")) 
    csp.add_constraint(CourseSchedulingConstraint("HS103", "HS105"))
    csp.add_constraint(CourseSchedulingConstraint("HS103", "HS208"))
    csp.add_constraint(CourseSchedulingConstraint("HS104", "HS105"))

     
    for c_list in clash_list:
        for crs in c_list:
            for drs in c_list[(c_list.index(crs)+1):]:
                if crs != "" and drs != "":
                    csp.add_constraint(CourseSchedulingConstraint(crs,drs))

    instructor_to_courses_map = {}
    for item in course_instructor_list:
        courseCode = item[0]
        instructor = item[1]
    
        if instructor in instructor_to_courses_map:
            temp_list = instructor_to_courses_map[instructor]
            
            temp_list = temp_list + [courseCode]
            instructor_to_courses_map[instructor] = temp_list
        else:
            temp_list = [courseCode]
            instructor_to_courses_map[instructor] = temp_list

    for instructor in instructor_to_courses_map:
        c_list = instructor_to_courses_map[instructor]
        for crs in c_list:
            for drs in c_list[(c_list.index(crs)+1):]:
                csp.add_constraint(CourseSchedulingConstraint(crs,drs))

    solution: Optional[Dict[str, str]] = csp.backtracking_search()
    if solution is None:
        print("No solution found!")
    else:
        slots_courses: Dict[str, List[str]] = {}
        for course in variables:
            if solution[course] in slots_courses.keys():
                slots_courses[solution[course]].append(course)
            else:
                slots_courses[solution[course]] = [course]
        print("Course Slots: ")
        for key, value in slots_courses.items():
            print(key, ' : ', value)
        
        classrooms = ["A1", "A2", "A3", "A4", "A5","A6","A7","A8", "A9"]
        course_classroom: Dict[str, str] = {}
        for slot in slots_courses.keys():
            i = 0
            for course in slots_courses[slot]:
                course_classroom[course] = classrooms[i]
                i = i + 1
        print("\nClassroom Allotment: ")
        for key, value in course_classroom.items():
            print(key, ' : ', value)
        

Course Slots: 
A  :  ['HS102', 'HS104', 'HS151', 'HS152', 'HS202', 'HS203', 'HS204', 'HS206', 'HS208']
B  :  ['HS103', 'HS205', 'HS235', 'HS254', 'HS255', 'HS261', 'HS301', 'HS304', 'HS341']
C  :  ['HS105', 'HS342', 'HS343', 'HS352', 'HS353', 'HS354', 'HS355', 'HS362', 'HS381']
D  :  ['HS351', 'HS363', 'HS372', 'HS373', 'HS382', 'HS392', 'HS403', 'HS450', 'HS502']
E  :  ['HS391', 'HS510']

Classroom Allotment: 
HS102  :  A1
HS104  :  A2
HS151  :  A3
HS152  :  A4
HS202  :  A5
HS203  :  A6
HS204  :  A7
HS206  :  A8
HS208  :  A9
HS103  :  A1
HS205  :  A2
HS235  :  A3
HS254  :  A4
HS255  :  A5
HS261  :  A6
HS301  :  A7
HS304  :  A8
HS341  :  A9
HS105  :  A1
HS342  :  A2
HS343  :  A3
HS352  :  A4
HS353  :  A5
HS354  :  A6
HS355  :  A7
HS362  :  A8
HS381  :  A9
HS351  :  A1
HS363  :  A2
HS372  :  A3
HS373  :  A4
HS382  :  A5
HS392  :  A6
HS403  :  A7
HS450  :  A8
HS502  :  A9
HS391  :  A1
HS510  :  A2
